In [9]:
import tifffile as tf
from roifile import ImagejRoi
import numpy as np
import os
import glob
from PIL import Image
#import matplotlib.pyplot as plt
# import torch
# import cv2
# import random
# import math

In [10]:
# Define 3 folders, the tif file directory with each image, the text file designation spot, the npz destination with \\
directory = #path to images
end_directory = #output path
npz_folder = #path to folder with npz outputs from MIA.

# Tif to txt + png

In [13]:
def obtain_coordinates(image_fp):
    try:
        roi = ImagejRoi.fromfile(image_fp)[0]
        points = roi.coordinates()
        return points
    except Exception as e:
        # If a Error is raised, print
        #print(f"Warning: {e}. File: {image_fp}")
        return  # 
    
    
def to_channel_3(numpy_image, img_path):
    # numpy_image of shape (2048, 2448, 3)
    if numpy_image.ndim == 3 and numpy_image.shape[2] == 3:
        return numpy_image[:,:,2]
    else:
        print("MISSING CHANNELS:",img_path)
        return numpy_image
    

    
def save_img_with_PIL(image_array: np.ndarray, savepath):
    try:
        # print("rr", image_array.shape)
        img = Image.fromarray(image_array)
        img.save(fp = savepath, compress_level=0)
        # out_img = Image.open(savepath)
        # out_img = np.array(out_img)
        # print("rd", out_img.shape)
        # print(np.sum(image_array == out_img), np.sum(image_array != out_img))
    except:
        print("SAVING ERROR:", savepath)
        
        

def round_and_save_coordinates_to_file(coordinates: np.ndarray, savepath):
    coordinates = np.round(coordinates)
    np.savetxt(savepath, coordinates, fmt='%d')
    # loaded_coordinates = np.loadtxt(savepath, dtype=int)
    # print(np.sum(np.equal(coordinates.astype(int), loaded_coordinates)),np.sum(~np.equal(coordinates.astype(int), loaded_coordinates)) )
    # print(coordinates[:5], "\nloaded:",loaded_coordinates[:5])



def process_raw_images(directory):
    #i=0
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".tif"):
                img_path = os.path.join(root, file)
                #print("image source", img_path)
                coordinates_use = obtain_coordinates(img_path)
                if isinstance(coordinates_use, np.ndarray):
                    
                    round_and_save_coordinates_to_file(coordinates_use, end_directory+file[:-4]+".txt")
                
                #if the image is in color un-comment this
                    image = tf.imread(img_path)
                    image = to_channel_3(image, img_path)
                    save_img_with_PIL(image, end_directory+file[:-4]+".png")                    
                else:
                    print("FAILED TO PROCESS COORDS:", img_path)
                    #i=i+1

                if False:
                    print("Failed image source", img_path)
    #print(i)

process_raw_images(directory)

FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_DMSO_104_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_DMSO_111_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_DMSO_13_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_DMSO_49_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_DMSO_6_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_DMSO_88_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_ctrl_100_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/COLO320DM_qPCR_JQ1_24h_ctrl_101_Merge.tif
FAILED TO PROCESS COORDS: /work/users/a/a/aarav7/Mask

# Txt to npz

In [18]:
end_directory = #path to folder with single pixel coords
npz_folder = #path to output folder for segmentation maps.

In [19]:
#transform txt files to npz files for segmentation
#creates a 7 pixel peak height and wide patch around the coordinates for class type
def validate_and_parse_coordinates(lines):
    coords = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 2:
            try:
                coords.append(tuple(map(int, parts)))
            except ValueError:
                continue  # Skip lines with non-integer values
    return coords

#seven pixels tall and wide diamond patch
def augment_coordinates(x, y):
    return [(x + dx, y + dy) for dx, dy in [(-3, 0), (-2, 1), (-1, 2), (0, 3), (1, 2), (2, 1), 
                                            (3, 0), (2, -1), (1, -2), (0, -3), (-1, -2), (-2, -1)]]

def create_combined_npz_from_coords(coords, file_name):
    npz_dict = {}
    arr_1 = np.array([1, -1]) #remove the dtype for this and check
    arr_3 = np.array([], dtype=np.float64)


    for i, (x, y) in enumerate(coords):
        augmented_coords = np.array(augment_coordinates(x, y)).reshape(-1, 1, 2) #removed , dtype=np.int32
        prefix = i * 3

        if i == 0:
            arr_0 = np.array(12345678910, dtype=np.int64)
            npz_dict[f'arr_{prefix}'] = arr_0
        else:
            arr_0 = np.array([], dtype=np.float64)

        npz_dict[f'arr_{prefix + 1}'] = arr_1
        npz_dict[f'arr_{prefix + 2}'] = augmented_coords
        npz_dict[f'arr_{prefix + 3}'] = arr_3

    np.savez(file_name, **npz_dict)

    
def process_directory(directory, npz_file_output):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                with open(os.path.join(root, file), 'r') as text_file:
                    lines = text_file.readlines()
                coords = validate_and_parse_coordinates(lines)
                #npz_file_path = file.rsplit('.', 1)[0] + '.npz'
                npz_file_path = npz_file_output + file.rsplit('.', 1)[0] + '.npz'
                create_combined_npz_from_coords(coords, npz_file_path)
                
                text_path = os.path.join(root, file)
                print(f"Processed: {text_path} to {npz_file_path}")



process_directory(end_directory[:], npz_folder)

Processed: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/coords/COLO320DM_qPCR_JQ1_24h_DMSO_104_Merge.txt to /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/Segmentation_labelsCOLO320DM_qPCR_JQ1_24h_DMSO_104_Merge.npz
Processed: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/coords/COLO320DM_qPCR_JQ1_24h_DMSO_111_Merge.txt to /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/Segmentation_labelsCOLO320DM_qPCR_JQ1_24h_DMSO_111_Merge.npz
Processed: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/coords/COLO320DM_qPCR_JQ1_24h_DMSO_13_Merge.txt to /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/Segmentation_labelsCOLO320DM_qPCR_JQ1_24h_DMSO_13_Merge.npz
Processed: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/coords/COLO320DM_qPCR_JQ1_24h_DMSO_49_Merge.txt to /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/Segmentation_labelsCOLO320DM_qPCR_JQ1_24h_DMSO_49_Merge.npz
Processed: /work/users/a/a/aarav7/Masked_img_3_and_4_1164imgs/coords/COLO320DM_qPCR_JQ1_24h_DMSO_6_M

# Check a output (only for small number of files)

In [19]:
npz_folder = #output folder
print('npz folder checked:', npz_folder)
for root, dirs, files in os.walk(npz_folder[:-2]):
    for file in files:
        if file.endswith(".npz"):
            file_path = os.path.join(root, file)
            data_display = np.load(file_path)
            arrays_display_expanded = {key: data_display[key] for key in data_display.keys()}
            print(arrays_display_expanded)


npz folder checked: C:\Users\koand\Downloads\Check up Tiff\Segmentation_labels\\
{'arr_0': array(12345678910, dtype=int64), 'arr_1': array([ 1, -1]), 'arr_2': array([[[767, 620]],

       [[768, 621]],

       [[769, 622]],

       [[770, 623]],

       [[771, 622]],

       [[772, 621]],

       [[773, 620]],

       [[772, 619]],

       [[771, 618]],

       [[770, 617]],

       [[769, 618]],

       [[768, 619]]]), 'arr_3': array([], dtype=float64), 'arr_4': array([ 1, -1]), 'arr_5': array([[[773, 642]],

       [[774, 643]],

       [[775, 644]],

       [[776, 645]],

       [[777, 644]],

       [[778, 643]],

       [[779, 642]],

       [[778, 641]],

       [[777, 640]],

       [[776, 639]],

       [[775, 640]],

       [[774, 641]]]), 'arr_6': array([], dtype=float64), 'arr_7': array([ 1, -1]), 'arr_8': array([[[792, 779]],

       [[793, 780]],

       [[794, 781]],

       [[795, 782]],

       [[796, 781]],

       [[797, 780]],

       [[798, 779]],

       [[797, 778

In [5]:
#GPT

import os
from collections import defaultdict

def get_file_names_without_extension(directory):
    """
    Get a set of file names without extensions from a directory.
    """
    files = os.listdir(directory)
    file_names = set()
    for file in files:
        file_name, _ = os.path.splitext(file)
        file_names.add(file_name)
    return file_names

def find_differences_and_repeats(dir1, dir2):
    """
    Compare the file names (without extensions) in two directories and find differences and repeats.
    """
    files_dir1 = get_file_names_without_extension(dir1)
    files_dir2 = get_file_names_without_extension(dir2)

    # Find files that are in dir1 but not in dir2
    unique_to_dir1 = files_dir1 - files_dir2
    # Find files that are in dir2 but not in dir1
    unique_to_dir2 = files_dir2 - files_dir1

    # Find repeats (files with the same name but different extensions)
    repeats = defaultdict(list)
    for file in os.listdir(dir1):
        file_name, _ = os.path.splitext(file)
        repeats[file_name].append(file)
    for file in os.listdir(dir2):
        file_name, _ = os.path.splitext(file)
        repeats[file_name].append(file)
    
    repeated_files = [file_name for file_name, instances in repeats.items() if len(instances) > 2]

    # Print results
    if unique_to_dir1:
        print("Files unique to directory 1:")
        for file in unique_to_dir1:
            print(file)

    if unique_to_dir2:
        print("Files unique to directory 2:")
        for file in unique_to_dir2:
            print(file)
    
    if repeated_files:
        print("Repeated files with different extensions:")
        for file in repeated_files:
            print(file)
    else:
        print("No repeated files with different extensions found.")

# Example usage
dir1 = #image directory
dir2 = #npz directory

find_differences_and_repeats(dir1, dir2)


Files unique to directory 1:
Tifs
Masks
npz
Coords
No repeated files with different extensions found.
